In [6]:
import pandas as pd
import numpy as np
from siuba import *
from plotnine import *
import plotly.express as px

In [92]:
df = pd.read_parquet("../data/preprocessed/accidents_from_mapper.parquet")
df = (
    df
    >> mutate(part=if_else(_.location <= 60.4, 'arava_dromit', 'arava_tihona'),
        after=_.year >= 2012
    )
    >> mutate(section=_.part + if_else(_.after, '_after_2012', '_before_2012'))
)

In [48]:
junctions = pd.read_csv("../data/preprocessed/junctions.csv")
junctions

,name,location
0,Eilot,11.9
1,Meches,15.6
2,Beer Ora,27.0
3,Timna,36.0
4,Samar,41.3
5,Yotvata,49.5
6,Grofit,54.0
7,Ktora,60.4
8,Yahel,72.3
9,Menuha,97.6


In [78]:
df

,year,month,day of week,day/night,accident_severity,accident_type,killed,severly_injured,lightly_injured,injured_pedestrians,...,localization_quality,settlement,road1,location,road2,road3,road4,part,after,part_time
0,2003,אפריל,ראשון,לילה,קשה,התנגשות חזית בצד,0,1,8,0,...,עיגון מדויק,NaN,90,100.3,NaN,NaN,NaN,arava_tihona,False,arava_tihona_before_2012
1,2003,מאי,שישי,לילה,קלה,התהפכות,0,0,3,0,...,עיגון מדויק,NaN,90,75.1,NaN,NaN,NaN,arava_tihona,False,arava_tihona_before_2012
2,2003,מאי,שבת,לילה,קלה,התהפכות,0,0,4,0,...,עיגון מדויק,NaN,90,39.9,NaN,NaN,NaN,arava_dromit,False,arava_dromit_before_2012
3,2003,יוני,חמישי,לילה,קלה,התנגשות חזית באחור,0,0,2,0,...,עיגון מדויק,NaN,90,135.8,NaN,NaN,NaN,arava_tihona,False,arava_tihona_before_2012
4,2003,מרס,שני,לילה,קטלנית,פגיעה בהולך רגל,1,4,22,1,...,עיגון מדויק,NaN,90,43.0,NaN,NaN,NaN,arava_dromit,False,arava_dromit_before_2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,2021,אפריל,שלישי,יום,קשה,התהפכות,0,3,0,0,...,עיגון מדויק,NaN,90,131.0,NaN,NaN,NaN,arava_tihona,True,arava_tihona_after_2012
344,2021,אפריל,רביעי,לילה,קטלנית,התנגשות חזית בחזית,2,0,1,0,...,עיגון מדויק,NaN,90,118.5,NaN,NaN,NaN,arava_tihona,True,arava_tihona_after_2012
345,2022,מרס,ראשון,יום,קלה,התנגשות חזית בצד,0,0,4,0,...,עיגון מדויק,NaN,90,NaN,1093.0,NaN,NaN,arava_tihona,True,arava_tihona_after_2012
346,2022,מרס,שבת,לילה,קלה,התנגשות עם עצם דומם,0,0,4,0,...,עיגון מדויק,NaN,90,38.9,NaN,NaN,NaN,arava_dromit,True,arava_dromit_after_2012


In [93]:
casualty_cols = [
    'killed',
    'severly_injured',
    'lightly_injured',
    'injured_pedestrians',
    'casualties_ages_0-19',
    'casualties_ages_20-64',
    'casualties_ages_65_plus',
    'total_casualties',
    'vehicle_count',
    'drivers',
]
summary = df.groupby('section')[casualty_cols].sum()
summary

,killed,severly_injured,lightly_injured,injured_pedestrians,casualties_ages_0-19,casualties_ages_20-64,casualties_ages_65_plus,total_casualties,vehicle_count,drivers
section,,,,,,,,,,
arava_dromit_after_2012,9,13,114,2,35,95,4,136,76,76
arava_dromit_before_2012,21,32,320,1,96,268,7,373,181,181
arava_tihona_after_2012,24,71,229,3,60,217,38,324,153,153
arava_tihona_before_2012,33,62,436,1,188,326,16,531,219,219


In [95]:
df.value_counts('section')

section
arava_tihona_before_2012    125
arava_dromit_before_2012     89
arava_tihona_after_2012      84
arava_dromit_after_2012      50
dtype: int64

In [96]:
df[casualty_cols].sum()

killed                       87
severly_injured             178
lightly_injured            1099
injured_pedestrians           7
casualties_ages_0-19        379
casualties_ages_20-64       906
casualties_ages_65_plus      65
total_casualties           1364
vehicle_count               629
drivers                     629
dtype: int64

In [111]:
fig = (
    df
    # >> filter(_.part=='south_')
    # >> filter(_.location >= 60.4, _.location <= 180)
    >> group_by(_.section)
    >> count(_.accident_type)
    >> pipe(lambda x: px.bar(x, x='accident_type', y='n', color='section', barmode='group',
        title='סוג תאונה בחלוקה למקטעים ותקופות', labels={'accident_type':'סוג תאונה', 'n': 'מספר תאונות'},
        category_orders={'section': ['arava_tihona_before_2012', 'arava_tihona_after_2012', 'arava_dromit_before_2012', 'arava_tihona_after_2012']}))
    # >> group_by(_.year)
    # >> summarize(killed=_.killed.sum(), lightly_injured=_.lightly_injured.sum())
)

fig.update_layout(
    legend=dict(
        yanchor='top',
        y=0.96,
        xanchor='right',
        x=0.99
    )
)
fig.show()

In [115]:
df.groupby('accident_type').sum()[casualty_cols].to_excel("acc_type.xlsx")

In [73]:

(
    df
    >> mutate(part=if_else(_.location <= 60.4, 'south', 'mid'),
        after=_.year >= 2012
    )
    >> mutate(part_time=_.part + if_else(_.after, '_after', '_before'))
    >> filter(_.part=='mid')
    # >> filter(_.location >= 60.4, _.location <= 180)
    >> group_by(_.part_time)
    >> count(_.accident_type)
    >> pipe(lambda x: px.bar(x, x='accident_type', y='n', color='part_time', barmode='group'))
    # >> group_by(_.year)
    # >> summarize(killed=_.killed.sum(), lightly_injured=_.lightly_injured.sum())
)

# Fatality of accidents

In [117]:
df.columns.tolist()

['year',
 'month',
 'day of week',
 'day/night',
 'accident_severity',
 'accident_type',
 'killed',
 'severly_injured',
 'lightly_injured',
 'injured_pedestrians',
 'casualties_ages_0-19',
 'casualties_ages_20-64',
 'casualties_ages_65_plus',
 'total_casualties',
 'vehicle_count',
 'drivers',
 'road_type',
 'localization_quality',
 'settlement',
 'road1',
 'location',
 'road2',
 'road3',
 'road4',
 'part',
 'after',
 'section']

In [124]:
df.groupby('section').accident_severity.value_counts(normalize=True)

section                   accident_severity
arava_dromit_after_2012   קלה                  0.660000
                          קשה                  0.220000
                          קטלנית               0.120000
arava_dromit_before_2012  קלה                  0.741573
                          קטלנית               0.157303
                          קשה                  0.101124
arava_tihona_after_2012   קלה                  0.392857
                          קשה                  0.380952
                          קטלנית               0.226190
arava_tihona_before_2012  קלה                  0.528000
                          קשה                  0.280000
                          קטלנית               0.192000
Name: accident_severity, dtype: float64